<a href="https://colab.research.google.com/github/RapaDewan/CSE445/blob/main/CSE499B_Trying_BERT/CSE499B_Trying_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Replace 'path_to_dataset' with the actual path to your dataset in Google Drive
import pandas as pd
dataset_path = '/content/drive/MyDrive/Resume.csv'
df = pd.read_csv(dataset_path)

In [13]:
from sklearn.utils import shuffle
# Shuffle the dataset
df = shuffle(df)

# Take 20 samples from the shuffled dataset
sample_df = df.head(20)

# Display the first few rows of the sample dataset
print(sample_df.head())

            ID                                         Resume_str  \
1162  38399177           CONSULTANT       Summary    Transitio...   
1354  22452756           SERVICES ENGINEER           Summary  ...   
1606  56151548           SUBSTITUTE TEACHER           Summary ...   
3     27018550           HR SPECIALIST       Summary    Dedica...   
2192  21856577           BRANCH BANKER III       Accomplishmen...   

                                            Resume_html    Category  
1162  <div class="fontsize fontface vmargins hmargin...  CONSULTANT  
1354  <div class="fontsize fontface vmargins hmargin...  AUTOMOBILE  
1606  <div class="fontsize fontface vmargins hmargin...     APPAREL  
3     <div class="fontsize fontface vmargins hmargin...          HR  
2192  <div class="fontsize fontface vmargins hmargin...     BANKING  


In [15]:
# Import necessary libraries
import re
from transformers import BertTokenizer

# Preprocessing function to clean the resume text
def preprocess_resume(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Preprocess the 'Resume_str' column in the sample data
sample_df['Processed_Resume'] = sample_df['Resume_str'].apply(preprocess_resume)

# Display the cleaned text
sample_df[['Resume_str', 'Processed_Resume']].head()


<ipython-input-15-ca8f1f7acf1f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Processed_Resume'] = sample_df['Resume_str'].apply(preprocess_resume)


,Resume_str,Processed_Resume
1162,CONSULTANT Summary Transitio...,CONSULTANT Summary Transitioning Mili...
1354,SERVICES ENGINEER Summary ...,SERVICES ENGINEER Summary Seeking...
1606,SUBSTITUTE TEACHER Summary ...,SUBSTITUTE TEACHER Summary Seekin...
3,HR SPECIALIST Summary Dedica...,HR SPECIALIST Summary Dedicated Drive...
2192,BRANCH BANKER III Accomplishmen...,BRANCH BANKER III Accomplishments M...


In [16]:
from transformers import BertTokenizer, BertModel
import torch

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for the resume text
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Get mean of the token embeddings
    return embeddings

# Function to calculate cosine similarity between two vectors
from sklearn.metrics.pairwise import cosine_similarity

# Define job description (Example: skills, experiences you're looking for)
job_description = "We are looking for a software engineer with experience in Python, machine learning, and AI."

# Preprocess job description
job_desc_processed = preprocess_resume(job_description)

# Get BERT embeddings for job description and each resume
job_desc_embedding = get_bert_embeddings(job_desc_processed)

# Calculate similarity for each resume
sample_df['Similarity_Score'] = sample_df['Processed_Resume'].apply(
    lambda x: cosine_similarity(get_bert_embeddings(x), job_desc_embedding).flatten()[0]
)

# Rank the CVs based on similarity scores
sample_df['Ranking'] = sample_df['Similarity_Score'].rank(ascending=False, method='min')

# Sort by ranking
sample_df = sample_df.sort_values(by='Ranking')

# Display the ranked CVs with their scores
print(sample_df[['ID', 'Ranking', 'Similarity_Score']])


            ID  Ranking  Similarity_Score
1807  21629057      1.0          0.657404
2327  37751611      2.0          0.649489
398   51209395      3.0          0.648690
1162  38399177      4.0          0.631002
315   10265057      5.0          0.627308
1213  13569152      6.0          0.623552
3     27018550      7.0          0.623187
2375  38355831      8.0          0.621056
776   25974844      9.0          0.620573
1354  22452756     10.0          0.616936
122   90066849     11.0          0.615886
983   69097572     12.0          0.615178
1606  56151548     13.0          0.606002
181   32532982     14.0          0.605949
1035  10603337     15.0          0.602842
2336  18586076     16.0          0.597783
2192  21856577     17.0          0.594464
453   90468982     18.0          0.585812
586   22423839     19.0          0.579298
1973  27835351     20.0          0.561306


<ipython-input-16-a72af2934198>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Similarity_Score'] = sample_df['Processed_Resume'].apply(
<ipython-input-16-a72af2934198>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['Ranking'] = sample_df['Similarity_Score'].rank(ascending=False, method='min')


In [18]:
# Function to generate feedback for each CV
def generate_feedback(row):
    feedback = f"Rank {row['Ranking']} - This CV matches the job description with a similarity score of {row['Similarity_Score']:.2f}."
    if row['Similarity_Score'] > 0.8:
        feedback += " The candidate seems to have the required skills and experience."
    elif row['Similarity_Score'] > 0.5:
        feedback += " The candidate has some relevant skills but may require more experience in certain areas."
    else:
        feedback += " The candidate seems to have less relevant experience or skills."
    return feedback

# Apply the feedback generation
sample_df['Feedback'] = sample_df.apply(generate_feedback, axis=1)

# Display the final output
final_output = sample_df[['ID', 'Ranking', 'Similarity_Score', 'Feedback']]
final_output


,ID,Ranking,Similarity_Score,Feedback
1807,21629057,1.0,0.657404,Rank 1.0 - This CV matches the job description...
2327,37751611,2.0,0.649489,Rank 2.0 - This CV matches the job description...
398,51209395,3.0,0.648690,Rank 3.0 - This CV matches the job description...
1162,38399177,4.0,0.631002,Rank 4.0 - This CV matches the job description...
315,10265057,5.0,0.627308,Rank 5.0 - This CV matches the job description...
1213,13569152,6.0,0.623552,Rank 6.0 - This CV matches the job description...
3,27018550,7.0,0.623187,Rank 7.0 - This CV matches the job description...
2375,38355831,8.0,0.621056,Rank 8.0 - This CV matches the job description...
776,25974844,9.0,0.620573,Rank 9.0 - This CV matches the job description...
1354,22452756,10.0,0.616936,Rank 10.0 - This CV matches the job descriptio...
